In [1]:
from joblib import Parallel, delayed

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt
import numpy as np
from gensim.models.ldamulticore import LdaMulticore
import pandas as pd 
import pyLDAvis
import pyLDAvis.gensim

# Tokeniza cada texto en una lista de palabras
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations



In [2]:
#dataset_name = sys.argv[1]
#df = pd.read_csv(dataset_name)
df = pd.read_csv("datosProcesados.csv")


# Seleccionar las filas en las que la opinion sea positiva
X = df[df['__target__'] == 2]
X = X[X['airline'] == 'US Airways']


# Paso todos los textos a una lista
textos = X.text.values.tolist()

# Paso cada texto de cada tweet de una string a una lista de palabras
data_words = list(sent_to_words(textos))

# Se crea el diccionario de las palabras; cada palabra unica contiene un identificador. Sirve para crear el corpus
id2word = corpora.Dictionary(data_words)

# Se crea el corpus
corpus = [id2word.doc2bow(text) for text in data_words]
#print(corpus[0])
# Cada palabra: (word_id, word_frequency). Si es (47,3) quiere decir que la palabra con id 47 aparece 3 veces en el documento




In [3]:
lda_model = LdaMulticore(corpus=corpus,  
    id2word=id2word,
    num_topics=5, 
    random_state=100,
    chunksize=100,
    passes=20,
    iterations=100,
    eval_every= None,
    per_word_topics=True,
    workers=8)
    
# La idea es tener la mejor similitud entre los textos de un topico mientras que se minimiza la similitud con los textos de otros
# La coherencia relaciona la distancia intracluster con la distancia intercluster
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


# Imprime los topicos; por cada topico muestra su id y luego las palabras mas frecuentes con la frecuencia de esa palabra en ese topico
print(lda_model.print_topics())

[(0, '0.062*"usairway" + 0.019*"thank" + 0.014*"great" + 0.014*"us" + 0.013*"servic" + 0.010*"home" + 0.010*"pilot" + 0.010*"flight" + 0.008*"keep" + 0.008*"thing"'), (1, '0.118*"usairway" + 0.085*"thank" + 0.033*"flight" + 0.012*"servic" + 0.012*"today" + 0.010*"made" + 0.009*"help" + 0.008*"custom" + 0.008*"great" + 0.007*"ok"'), (2, '0.092*"usairway" + 0.049*"thank" + 0.019*"great" + 0.016*"flight" + 0.013*"help" + 0.011*"servic" + 0.011*"us" + 0.010*"appreci" + 0.010*"work" + 0.009*"today"'), (3, '0.063*"usairway" + 0.021*"thank" + 0.016*"us" + 0.015*"time" + 0.014*"flight" + 0.012*"get" + 0.010*"awesom" + 0.010*"gate" + 0.009*"crew" + 0.009*"agent"'), (4, '0.080*"usairway" + 0.028*"follow" + 0.025*"pleas" + 0.024*"get" + 0.022*"thank" + 0.019*"flight" + 0.019*"best" + 0.017*"back" + 0.013*"amp" + 0.011*"hope"')]


In [4]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.069123  0.075516       1        1  33.739875
1      0.000476 -0.108325       2        1  26.606310
4     -0.119620 -0.031360       3        1  15.381216
3     -0.050410  0.101198       4        1  12.203998
0      0.100432 -0.037029       5        1  12.068601, topic_info=       Term        Freq       Total Category  logprob  loglift
243  follow   10.000000   10.000000  Default  30.0000  30.0000
239   pleas   12.000000   12.000000  Default  29.0000  29.0000
23      get   21.000000   21.000000  Default  28.0000  28.0000
61     back    7.000000    7.000000  Default  27.0000  27.0000
4     thank  107.000000  107.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
28   flight    2.820425   45.818600   Topic5  -4.5783  -0.6732
23      get    2.154530   21.781452   Topic5  -4.8476  -0.1989
113  attend    1.482424    7.524121   Topic5  -5.2215   0.4901
39     amaz    1.482001    7.787975   Topic5  -5.2218   0.4554
79      fli    1.481970    7.436889   Topic5  -5.2218   0.5015

[332 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
554       5  0.715306        ab
205       1  0.226788   absolut
205       2  0.453576   absolut
205       4  0.226788   absolut
0         2  0.886282  accommod
...     ...       ...       ...
106       1  0.601670      work
106       2  0.171906      work
106       3  0.085953      work
106       5  0.085953      work
559       5  0.715306      year

[463 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 4, 1])